In [1]:
from netcal.binning.BBQ import BBQ
from netcal.binning.HistogramBinning import HistogramBinning
from netcal.binning.IsotonicRegression import IsotonicRegression
from netcal.scaling import TemperatureScaling
from netcal.metrics import ECE
from numpy import random
random.rand(4)

def traditional_calibrator(link_logits, link_labels,  link_logits_test, tipo='iso'):
    if tipo=='iso':
        lr = IsotonicRegression()
        lr.fit(link_logits, link_labels)
        lr_test_predictions = lr.transform(link_logits_test)
    elif tipo=='temp':
        lr = TemperatureScaling()
        lr.fit(link_logits, link_labels)
        lr_test_predictions = lr.transform(link_logits_test)
    elif tipo=='bbq':
        lr = BBQ()
        lr.fit(link_logits, link_labels)
        lr_test_predictions = lr.transform(link_logits_test)
    elif tipo=='hist':
        lr = HistogramBinning(bins=16, equal_intervals=True)
        lr.fit(link_logits, link_labels)
        lr_test_predictions = lr.transform(link_logits_test)
    
    return lr_test_predictions

2023-08-11 08:45:16.625573: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-11 08:45:16.658118: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-11 08:45:16.807615: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-11 08:45:16.808336: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-11 08:45:17.413681: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
import torch
from utils import expected_calibration_error, plot_reliability_diagram, get_link_labels, accuracy
from utils_calib import ECELoss
from models import DeepVGAE
from torch_geometric.utils import negative_sampling
from torch_geometric.datasets.planetoid import Planetoid
import torch_geometric.transforms as T
from torch_geometric.utils import train_test_split_edges
import os
import numpy as np

device_string = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device = torch.device(device_string)

seed = 42
dataset_name = "cora"
gnn = 'VGAE'
path="models/"

os.makedirs("datasets", exist_ok=True)
dataset = Planetoid("datasets", dataset_name, transform=T.NormalizeFeatures())
data = dataset[0].to(device)
all_edge_index = data.edge_index
data = train_test_split_edges(data, 0.05, 0.1)

data = data.to(device)

enc_in_channels = data.x.shape[1]
enc_hidden_channels = 32
enc_out_channels = 16


model = DeepVGAE(enc_in_channels, enc_hidden_channels, enc_out_channels).to(device)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
ckpt_destilado = torch.load(path + gnn + '_'+dataset_name+ '_seed_' + str(seed) + '.pth.tar', map_location=torch.device(device))
model.load_state_dict(ckpt_destilado["model_state"]) 

for para in model.parameters():
    para.requires_grad = False

/home/erik/.local/lib/python3.10/site-packages/torch_geometric/typing.py:25: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: /home/erik/.local/lib/python3.10/site-packages/libpyg.so: undefined symbol: _ZNK3c104impl13OperatorEntry24assertSignatureIsCorrectENS0_12CppSignatureEb
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/home/erik/.local/lib/python3.10/site-packages/torch_geometric/typing.py:70: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /home/erik/.local/lib/python3.10/site-packages/libpyg.so: undefined symbol: _ZNK3c104impl13OperatorEntry24assertSignatureIsCorrectENS0_12CppSignatureEb
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "
/home/erik/.local/lib/python3.10/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


In [3]:
model.eval()
prefix= "train"
pos_edge_index_train = data[f'{prefix}_pos_edge_index']
if prefix == 'train':
    neg_edge_index_train = negative_sampling(
        edge_index=data.train_pos_edge_index, #positive edges
        num_nodes=data.num_nodes, # number of nodes
        num_neg_samples=data.train_pos_edge_index.size(1)) # number of neg_sample equal to number of pos_edges
else:
    neg_edge_index = data[f'{prefix}_neg_edge_index']
link_logits_train = model.predict(data.x, pos_edge_index_train, neg_edge_index_train)
link_labels_train = get_link_labels(pos_edge_index_train, neg_edge_index_train, device) # get link
edge_index_train = torch.cat([pos_edge_index_train, neg_edge_index_train], dim=1)
n_train = edge_index_train.shape[1]


In [4]:
model.eval()
prefix= "val"

pos_edge_index_val = data[f'{prefix}_pos_edge_index']
neg_edge_index_val = data[f'{prefix}_neg_edge_index']

edge_index_val = torch.cat([pos_edge_index_val, neg_edge_index_val], dim=1)
n_val = edge_index_val.shape[1]


link_logits_val = model.predict(data.x, pos_edge_index_train, edge_index_val)[-n_val:]
link_labels_val = get_link_labels(pos_edge_index_val, neg_edge_index_val, device) # get link


prefix= "test"
pos_edge_index_test = data[f'{prefix}_pos_edge_index'].to(device)
neg_edge_index_test = data[f'{prefix}_neg_edge_index'].to(device)

edge_index_test = torch.cat([pos_edge_index_test, neg_edge_index_test], dim=1)
n_test = edge_index_test.shape[1]

link_logits_test = model.predict(data.x, pos_edge_index_train, edge_index_test)[-n_test:]
link_labels_test = get_link_labels(pos_edge_index_test, neg_edge_index_test, device) # get link

link_labels_val = link_labels_val.type(torch.LongTensor)
link_labels_test = link_labels_test.type(torch.LongTensor)

In [9]:
from ogb.linkproppred import Evaluator
from sklearn.metrics import roc_auc_score
from netcal.metrics import ECE
import numpy as np

eces_test = []
aucs_test = []

link_confidences_train = torch.cat([1-link_logits_train.unsqueeze(1).sigmoid(), link_logits_train.unsqueeze(1).sigmoid()], dim=1)
link_confidences_val = torch.cat([1-link_logits_val.unsqueeze(1).sigmoid(), link_logits_val.unsqueeze(1).sigmoid()], dim=1)
link_confidences_test = torch.cat([1-link_logits_test.unsqueeze(1).sigmoid(), link_logits_test.unsqueeze(1).sigmoid()], dim=1)

output_test = torch.tensor(traditional_calibrator(link_confidences_train.cpu().numpy(),
       link_labels_train.cpu().numpy(),
       link_confidences_test.cpu().numpy(),
        tipo='hist')).unsqueeze(1)



In [10]:
metric = ECE(15)
ece = metric.measure(torch.cat([1-output_test, output_test], dim=1).numpy(), link_labels_test.numpy())
print(f'ece: {round(ece,8)}')

ece: 0.06436632
